In [64]:
import tensorflow as tf
import numpy as np
import re
import pandas as pd
import nltk
import itertools
from tqdm import tqdm
!pip install --upgrade tensorlayer

from sklearn.utils import shuffle
import tensorlayer as tl
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Requirement already up-to-date: tensorlayer in /usr/local/lib/python3.6/dist-packages (1.11.1)
    100% |████████████████████████████████| 2.0MB 12.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  [PIL]
You must restart the runtime in order to use newly installed versions.
Found GPU at: /device:GPU:0


In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


from google.colab import drive


In [66]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
ls "drive/My Drive/Colab Notebooks/movie_lines.txt"

'drive/My Drive/Colab Notebooks/movie_lines.txt'


In [0]:
#lines =open('movie_lines.tsv', encoding='utf-8', errors='ignore').read().split('\n')
#conversations = open('movie_conversations.tsv', encoding='utf-8', errors='ignore').read().split('\n')

In [0]:
lines =open('drive/My Drive/Colab Notebooks/movie_lines.tsv', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('drive/My Drive/Colab Notebooks/movie_conversations.tsv', encoding='utf-8', errors='ignore').read().split('\n')

OUTPUT: lineID +++$+++ characterID +++$+++ movieID +++$+++ movieID +++$+++ text of the utterance

In [70]:
lines[:2]

['L1045\tu0\tm0\tBIANCA\tThey do not!', 'L1044\tu2\tm0\tCAMERON\tThey do to!']

OUTPUT: characterID +++$+++ characterID +++$+++ movieID +++$+++ [list of the utterances]

In [71]:
conversations[:2]

["u0\tu2\tm0\t['L194' 'L195' 'L196' 'L197']", "u0\tu2\tm0\t['L198' 'L199']"]

In [0]:
# Creating a dictionary that maps each line and its id
id2line = {}
for line in lines:
    _line = line.split('\t')
    if len(_line) >= 5:
        lid=_line[0].replace("'","").replace("\"","")
        id2line[lid] = ''.join(_line[4:(len(_line))])

# Creating a list of all of the conversations
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split('\t')[-1][1:-1].replace("'","").replace("[","").replace("]","")
    conversations_ids.append(_conversation.split(' '))

# Getting separately the questions and the answers
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

sample dialoges

In [12]:
for i in range(10):
        print("Question: ",questions[i])
        print("Answer: ",answers[i])
        print(" ")

Question:  Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Answer:  Well I thought we'd start with pronunciation if that's okay with you.
 
Question:  Well I thought we'd start with pronunciation if that's okay with you.
Answer:  Not the hacking and gagging and spitting part.  Please.
 
Question:  Not the hacking and gagging and spitting part.  Please.
Answer:  Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?
 
Question:  You're asking me out.  That's so cute. What's your name again?
Answer:  Forget it.
 
Question:  No no it's my fault -- we didn't have a proper introduction ---
Answer:  Cameron.
 
Question:  Cameron.
Answer:  The thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.
 
Question:  The thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until s

### Pre-procces , to-lower,replace words

In [0]:
def pre(text):
    text=text.lower()
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = re.sub(r"  ", " ", text)

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text  = re.sub(r"aren't", "are not",text)
    text  = re.sub(r"'bout", "about",text)
    text  = re.sub(r"'til", "until",text)
    text  = re .sub("mightn't", "might not",text)
    text  = re.sub(r"shouldn't", "should not",text)
    text  = re.sub(r"couldn't", "could not",text)
    text  = re.sub(r"didn't", "did not",text)
    text  = re.sub(r"doesn't", "does not",text)
    text  = re.sub(r"don't", "do not",text)
    text  = re.sub(r"hadn't", "had not",text)
    text  = re.sub(r"hasn't", "has not",text)
    text  = re.sub(r"haven't", "have not",text)
    text  = re.sub(r"he'd", "he had",text)
    text  = re.sub(r"he'll", "he will",text)
    text  = re.sub(r"I'd", "I had",text)
    text  = re.sub(r"isn't", "is not",text)
    text  = re.sub(r"let's", "let us",text)
    text  = re.sub(r"mightn't", "might not",text)
    text  = re.sub(r"mustn't", "must not",text)
    text  = re.sub(r"shan't", "shall not",text)
    text  = re.sub(r"there's", "there is",text)
    text  = re.sub(r"they'd", "they had",text)
    text  = re.sub(r"they'll", "they will",text)
    text  = re.sub(r"they're", "they are",text)
    text  = re.sub(r"they've", "they have",text)
#     text  = re.sub(r"she'd", "she had",text)
#     text  = re.sub(r"she'll", "she will",text)
#     text  = re.sub(r"she's", "she is",text)
#     text  = re.sub(r"shouldn't", "should not",text)
#     text  = re.sub(r"that's", "that is",text)

#     text  = re.sub(r"we'd", "we had",text)
#     text  = re.sub(r"we're", "we are",text)
#     text  = re.sub(r"we've", "we have",text)
#     text  = re.sub(r"weren't", "were not",text)
#     text  = re.sub(r"what'll", "what will",text)
#     text  = re.sub(r"what're", "what are",text)
#     text  = re.sub(r"what's", "what is",text)
#     text  = re.sub(r"what've", "what have",text)
#     text  = re.sub(r"where's", "where is",text)
#     text  = re.sub(r"who's", "who had",text)
#     text  = re.sub(r"who'll", "who will",text)
#     text  = re.sub(r"who're", "who are",text)
#     text  = re.sub(r"who's", "who is",text)
#     text  = re.sub(r"who've", "who have",text)
#     text  = re.sub(r"won't", "will not",text)
#     text  = re.sub(r"wouldn't", "would not",text)
#     text  = re.sub(r"you'd", "you had",text)
#     text  = re.sub(r"you'll", "you will",text)
#     text  = re.sub(r"you're", "you are",text)
#     text  = re.sub(r"you've", "you have",text)

    
    return text

In [0]:
c_answers=[]
c_questions=[]

for q in questions:
    c_answers.append(pre(q))
for a in answers:
    c_questions.append(pre(a))
    


In [75]:
c_answers[:10]

['can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part please',
 'you are asking me out that is so cute that is your name again',
 'no no it is my fault we did not have a proper introduction ',
 'cameron',
 'the thing is cameron i am at the mercy of a particularly hideous breed of loser my sister i cannot date until she does',
 'why',
 'unsolved mystery she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend']

In [76]:
c_questions[:10]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part please',
 'okay then how about we try out some french cuisine saturday night',
 'forget it',
 'cameron',
 'the thing is cameron i am at the mercy of a particularly hideous breed of loser my sister i cannot date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do']

### Statistics

In [77]:
#total_len

print(len(c_questions))
print(len(c_answers))


221616
221616


In [78]:
!pip install nltk

In [79]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
len_questions=[]
len_answers=[]

wq_occ= dict()
wa_occ = dict()

for q in c_questions:
    len_questions.append(len( nltk.word_tokenize(q)))
    for wq in nltk.word_tokenize(q):
        if wq in wq_occ:
            wq_occ[wq]+=1
        else:
            wq_occ[wq]=1
for a in c_answers:
    len_answers.append(len( nltk.word_tokenize(a)))
    for wa in nltk.word_tokenize(a):
        if wa in wa_occ:
            wa_occ[wa]+=1
        else:
            wa_occ[wa]=1





In [0]:
ans_lengths = pd.DataFrame(len_answers, columns=['lens'])
q_lengths = pd.DataFrame(len_answers, columns=['lens'])


In [82]:
#stats len
ans_lengths.describe()

,lens
count,221616.000000
mean,10.803859
std,11.928132
min,0.000000
25%,4.000000
50%,7.000000
75%,13.000000
max,319.000000


In [83]:
##stats answers
q_lengths.describe()

,lens
count,221616.000000
mean,10.803859
std,11.928132
min,0.000000
25%,4.000000
50%,7.000000
75%,13.000000
max,319.000000


In [84]:
##find most frequent words in Q&A
print("-----------------Q-------------------------------")

q_freq = sorted(wq_occ.items(), key=lambda kv: kv[1])
for i in range(len(q_freq)-1,len(q_freq)-10,-1):
    print(q_freq[i])
print("-----------------A-------------------------------")
    
a_freq = sorted(wa_occ.items(), key=lambda kv: kv[1])
for i in range(len(a_freq)-1,len(a_freq)-10,-1):
    print(a_freq[i])

-----------------Q-------------------------------
('i', 106654)
('you', 105326)
('the', 71110)
('to', 58921)
('is', 58168)
('not', 57333)
('a', 52334)
('it', 49363)
('that', 37475)
-----------------A-------------------------------
('you', 108839)
('i', 97918)
('the', 69543)
('is', 57631)
('to', 57267)
('not', 52888)
('a', 49700)
('it', 45592)
('that', 36992)


In [85]:
##find most infrequent words in Q&A
print("-----------------Q-------------------------------")

q_freq = sorted(wq_occ.items(), key=lambda kv: kv[1])
for i in range(0,10,1):
    print(q_freq[i])
print("-----------------A-------------------------------")
    
a_freq = sorted(wa_occ.items(), key=lambda kv: kv[1])
for i in range(0,10,1):
    print(a_freq[i])
    


-----------------Q-------------------------------
('pronunciation', 1)
('eber', 1)
('diffuser', 1)
('jared', 1)
('samesex', 1)
('clouted', 1)
('fen', 1)
('hedgepig', 1)
('mandella', 1)
('obsess', 1)
-----------------A-------------------------------
('roxanne', 1)
('korrine', 1)
('pronunciation', 1)
("c'esc", 1)
('tete', 1)
('guillermo', 1)
('90210', 1)
('bonchowski', 1)
('dooropening', 1)
('coatholding', 1)


In [0]:
wa= [i[1] for i in list(wa_occ.items())]
wq = [i[1] for i in list(wq_occ.items())]

In [0]:
wans_count = pd.DataFrame(wa, columns=['len'])
wq_count = pd.DataFrame(wq, columns=['len'])

In [88]:
wans_count.describe()

,len
count,54070.000000
mean,44.281635
std,1010.589969
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,108839.000000


In [89]:
wq_count.describe()

,len
count,54919.000000
mean,45.133597
std,1035.346891
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,106654.000000


In [90]:
print(len(c_answers),len(c_questions))

221616 221616


In [0]:
def short_qa(clean_questions,clean_answers,min_,max_):
    questions_valid_=[]
    answers_valid_ = []
    questions_short = []
    answers_short = []

    for i,question in enumerate(clean_questions):
        if len(question.split()) >= min_ and len(question.split()) <= max_:
            questions_valid_.append(question)
            answers_valid_.append(clean_answers[i])

    for i,answer in enumerate(answers_valid_):
        if len(answer.split()) >= min_ and len(answer.split()) <= max_:
            answers_short.append(answer)
            questions_short.append(questions_valid_[i])
    return (answers_short,questions_short)


In [92]:
(q,a)=short_qa(c_questions,c_answers,2,20)
print(len(q))

138186


In [93]:
# Compare the number of lines we will use with the total number of lines.
print("# of questions:", len(q))
print("# of answers:", len(a))
print("% of data used: {}%".format(round(len(q)/len(c_questions),4)*100))

# of questions: 138186
# of answers: 138186
% of data used: 62.35000000000001%


##Create Vocabulary 
+++> word->ids (wXXX2idx), XXX-> question or answer
+++> ids->word (idx2wXXX),  XXX-> question or answer

In [0]:
vocab = {}
for question in q:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in a:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [0]:
thers=2

wo2idx = {}
word_num=1
for word, count in vocab.items():
    if count >= thers:
        wo2idx[word] = word_num
        word_num+=1

In [96]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", word_num)

Size of total vocab: 45941
Size of vocab we will use: 29808


In [97]:
print(len(wo2idx))
print(len(q))
print(len(a))


29807
138186
138186


In [98]:
codes = ['EOS','UNK','GO']
wo2idx['PAD']=0
for code in codes:
    wo2idx[code] = len(wo2idx)
    print(wo2idx[code])
idx2word = {v_i: v for v, v_i in wo2idx.items()}

29808
29809
29810


In [99]:
word_count = 0
unk_count = 0

final_q = []
for question in q:
    qwos = []
    for word in question.split():
        if word not in wo2idx:
            qwos.append(wo2idx['UNK'])
            unk_count += 1
        else:
            qwos.append(wo2idx[word])
            word_count += 1
    final_q.append(qwos)

final_a = []
for answer in a:
    awos = []
    for word in answer.split():
        if word not in wo2idx:
            awos.append(wo2idx['UNK'])
            unk_count += 1
        else:
            awos.append(wo2idx[word])
            word_count += 1

    final_a.append(awos)

unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))


Total number of words: 2180175
Number of times <UNK> is used: 16134
Percent of words that are <UNK>: 0.74%


In [0]:
## we can also sort the questions and answers

def sorting(question_int,answers_int):
  
  # Sort questions and answers by the length of questions.
  # This will reduce the amount of padding during training
  # Which should speed up training and help to reduce the loss

  sorted_questions = []
  sorted_answers = []

  for length in range(1, max_line_length+1):
      for i in enumerate(questions_int):
          if len(i[1]) == length:
              sorted_questions.append(questions_int[i[0]])
              sorted_answers.append(answers_int[i[0]])

  print(len(sorted_questions))
  print(len(sorted_answers))
  print()
  for i in range(3):
      print(sorted_questions[i])
      print(sorted_answers[i])
      print()



In [101]:
print(final_a[0])
print(a[0])
print(final_q[0])
print(q[0])

[13, 14, 15, 16, 17, 16, 18, 19, 20]
not the hacking and gagging and spitting part please
[1, 2, 3, 4, 5, 6, 7, 29809, 8, 9, 10, 11, 7, 12]
well i thought we would start with pronunciation if that is okay with you


In [102]:
print(len(final_q),len(final_a))

138186 138186


In [0]:
from tensorlayer.layers import DenseLayer, EmbeddingInputlayer, Seq2Seq, retrieve_seq_length_op2

In [0]:
def create_model(encode_seqs, decode_seqs, src_vocab_size, emb_dim, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        with tf.variable_scope("embedding") as vs:
            #The input layer:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = src_vocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            vs.reuse_variables()
            #The output layer
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = src_vocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            
        net_rnn = Seq2Seq(net_encode, net_decode,
                cell_fn = tf.nn.rnn_cell.LSTMCell,
                n_hidden = emb_dim,
                initializer = tf.random_uniform_initializer(-0.1, 0.1),
                encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
                decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
                initial_state_encode = None,
                dropout = (0.5 if is_train else None),
                n_layer = 3,
                return_seq_2d = True,
                name = 'seq2seq')

        net_out = DenseLayer(net_rnn, n_units=src_vocab_size, act=tf.identity, name='output')
    return net_out, net_rnn

In [0]:
tf.reset_default_graph()
sess_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
sess_config.gpu_options.allow_growth = True

In [106]:
batch_size=64
n_step = len(final_q) // batch_size
with tf.device('/device:GPU:0'):
  encode_seqs_= tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
  decode_seqs_ = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
  target_seqs_ = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
  target_mask_ = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") 
  net_out, _ = create_model(encode_seqs_, decode_seqs_, len(wo2idx), 128, is_train=True, reuse=False)
  net_out.print_params(False)

  # Inference Data Placeholders
  encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
  decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")

  net, net_rnn = create_model(encode_seqs2, decode_seqs2, len(wo2idx), 128, is_train=False, reuse=True)
  y = tf.nn.softmax(net.outputs)

  # Loss Function
  loss = tl.cost.cross_entropy_seq_with_mask(logits=net_out.outputs, target_seqs=target_seqs_, 
                                              input_mask=target_mask_, return_details=False, name='cost')

  # Optimizer
  optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
  train_op = optimizer.minimize(loss)
  

[TL] EmbeddingInputlayer model/embedding/seq_embedding: (29811, 128)
[TL] EmbeddingInputlayer model/embedding/seq_embedding: (29811, 128)
[TL] [*] Seq2Seq model/seq2seq: n_hidden: 128 cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL] DynamicRNNLayer model/seq2seq/encode: n_hidden: 128, in_dim: 3 in_shape: (64, ?, 128) cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL]        batch_size (concurrent processes): 64
[TL] DynamicRNNLayer model/seq2seq/decode: n_hidden: 128, in_dim: 3 in_shape: (64, ?, 128) cell_fn: LSTMCell dropout: 0.5 n_layer: 3
[TL]        batch_size (concurrent processes): 64
[TL] DenseLayer  model/output: 29811 No Activation
[TL]   param   0: model/embedding/seq_embedding/embeddings:0 (29811, 128)       float32_ref
[TL]   param   1: model/seq2seq/encode/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0 (256, 512)         float32_ref
[TL]   param   2: model/seq2seq/encode/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0 (512,)             float32_ref
[TL]   param   3: model/seq2seq/encod

In [0]:
a=tf.initialize_all_variables()
b=tf.global_variables_initializer()
sess=tf.Session(config=sess_config)

In [109]:
start_id = wo2idx['GO']  
end_id =  wo2idx['EOS'] 
unk_id = wo2idx['UNK']
print(start_id,end_id,wo2idx['PAD'])

29810 29808 0


In [0]:
def inference(seed):
  seed=pre(seed)
  seed_id = [wo2idx.get(w, unk_id) for w in seed.split(" ")]
  
  state = sess.run(net_rnn.final_state_encode,
                  {encode_seqs2: [seed_id]})
  #Run the softmax function to the output of the decoded rnn used for inference
  o, state = sess.run([y, net_rnn.final_state_decode],
                  {net_rnn.initial_state_decode: state,
                  decode_seqs2: [[start_id]]})
  w_id = tl.nlp.sample_top(o[0], top_k=10)
  w = idx2word[w_id]
  # Decode and feed state iteratively
  sentence = [w]
  #As far as the output symbol is not EOS,keep feeding.
  for _ in range(30): # max sentence length
      o, state = sess.run([y, net_rnn.final_state_decode],
                      {net_rnn.initial_state_decode: state,
                      decode_seqs2: [[w_id]]})
      w_id = tl.nlp.sample_top(o[0], top_k=10)
      w = idx2word[w_id]
      if w_id == end_id:
          break
      sentence = sentence + [w]
  return sentence

In [0]:
import time
seeds = ["do you love trump","how are you","i love you","fuck you","do you like drinking","what is your name"]
num_epochs=10
sess.run(a)
sess.run(b)
import sys
max_=20
sys.stdout.flush()
for epoch in range(num_epochs):
    trainX, trainY = shuffle(final_q, final_a, random_state=0)
    start_time = time.time()

    total_loss, n_iter = 0, 0
    for X, Y in tqdm(tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False), 
                    total=n_step, desc='Epoch[{}/{}]'.format(epoch + 1, num_epochs), leave=False):
        

        
        
        
        X = tl.prepro.pad_sequences(X)
        _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
        _target_seqs = tl.prepro.pad_sequences(_target_seqs)
        _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
        _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
        _target_mask = tl.prepro.sequences_get_mask(_target_seqs)


        
        
        _, loss_iter = sess.run([train_op, loss], {encode_seqs_: X, decode_seqs_: _decode_seqs,
                        target_seqs_: _target_seqs, target_mask_: _target_mask})
        total_loss += loss_iter
        n_iter += 1
    end_time = time.time()

    batch_time = end_time - start_time

    # printing average loss after every epoch
    print('Epoch [{}/{}]: loss {:.4f} learning rate={:.4f}, Seconds: {:>5.2f}'.format(epoch + 1, num_epochs, total_loss / n_iter,optimizer._lr,batch_time))

    # inference after every epoch
    for seed in seeds:
        print("SEED====>", seed)
        for _ in range(5):
            sentence = inference(seed)
            print("RES=====>", ' '.join(sentence))


# session cleanup



Epoch[1/10]: 100%|██████████| 2159/2159 [05:08<00:00,  7.11it/s]
                                                                

Epoch [1/10]: loss 5.7304 learning rate=0.0010, Seconds: 308.44
SEED====> do you love trump
RES=====> what is you
RES=====> no that
RES=====> he would like me of a name you have the man is a way
RES=====> i did not think
RES=====> i are you get you
SEED====> how are you
RES=====> oh i would know
RES=====> i will get to you
RES=====> yeah i am to
RES=====> what i am to a good
RES=====> it is not it
SEED====> i love you
RES=====> you did not do
RES=====> no you have a good the
RES=====> you are sorry
RES=====> yeah he is you
RES=====> he do you know i know it are the man
SEED====> fuck you
RES=====> what i am not a good
RES=====> i would have a way for it
RES=====> it is not in him
RES=====> you can not have the little UNK
RES=====> i do not know
SEED====> do you like drinking
RES=====> i are just a little
RES=====> what
RES=====> it are not a lot i do not want to
RES=====> no you will be a UNK
RES=====> no what is my lot you have a UNK
SEED====> what is your name
RES=====> i cannot be
R


Epoch[2/10]:   0%|          | 0/2159 [00:00<?, ?it/s]

RES=====> i do not have you be to be you a car



Epoch[2/10]: 100%|██████████| 2159/2159 [05:08<00:00,  6.90it/s]
                                                                

Epoch [2/10]: loss 5.2548 learning rate=0.0010, Seconds: 308.61
SEED====> do you love trump
RES=====> yeah you will be to do with a little UNK
RES=====> you have not
RES=====> i think i am sorry
RES=====> no do not get
RES=====> that is you
SEED====> how are you
RES=====> you have
RES=====> i am sorry i did not
RES=====> no the minute
RES=====> i will get it
RES=====> i have going to be a man
SEED====> i love you
RES=====> i am sorry
RES=====> i am not not i do not want i do it to be a little man
RES=====> you are gonna do
RES=====> no the UNK and a little UNK and
RES=====> i will see him
SEED====> fuck you
RES=====> well i am sure
RES=====> you have just get your lot of
RES=====> i do not want to get a UNK i will not have it
RES=====> you were the little way
RES=====> what would you
SEED====> do you like drinking
RES=====> no i am sorry
RES=====> you know it
RES=====> i am just
RES=====> it is all the little UNK
RES=====> i do do
SEED====> what is your name
RES=====> you do not think 


Epoch[3/10]:  77%|███████▋  | 1670/2159 [03:57<01:10,  6.97it/s]

In [0]:
s=''
print('q to exit')
while s!='q':
  s=input('-')
  if s!='q':
    m=inference(s)
    print("-",' '.join(m))
print("Adios!")